# 기본 세팅

In [2]:
#로직 설정용 라이브러리 모음
import os
import sys
import numpy as np

import pandas as pd
pd.set_option('display.max_row',500)
pd.set_option('display.max_columns', 30)
from pandas import DataFrame

from collections import Counter

import time
from datetime import datetime, date, timedelta

import warnings
warnings.filterwarnings(action='ignore')

from tqdm import tqdm, tqdm_notebook

from itertools import product

#그래프 생성 라이브러리 모음
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import font_manager, rc #그래프 폰트 깨질경우 방지 부분
matplotlib.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] = False

#폰트 깨지는거 수정 부분
import platform
if platform.system()=='Windows':
    font_name=font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font',family=font_name)
else:
    rc('font',family='AppleGothic')

#머신러닝 수행 라이브러리 모음
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix

import lightgbm as lgbm # lightgbm 부스팅 알고리즘 사용
from lightgbm import LGBMClassifier
#from sklearn.ensemble import RandomForestClassifier
# from xgboost import XGBClassifier

#현재 데이터에서 발생하는 type error를 해결하기 위한 함수 생성
def coerce_df_columns_to_numeric(df, column_list):
    df[column_list] = df[column_list].apply(pd.to_numeric, errors='coerce')
    
    
print("현재 사용중인 파이썬 version은 " + sys.version + " 입니다.")
print("현재 사용중인 numpy version은 " + np.__version__ + " 입니다.")
print("현재 사용중인 pandas version은 " + pd.__version__ + " 입니다.")
print("현재 사용중인 matplotlib version은 " + matplotlib.__version__ + " 입니다.")
print("현재 사용중인 lgbm version은 " + lgbm.__version__ + " 입니다.")

현재 사용중인 파이썬 version은 3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)] 입니다.
현재 사용중인 numpy version은 1.19.5 입니다.
현재 사용중인 pandas version은 1.1.3 입니다.
현재 사용중인 matplotlib version은 3.3.2 입니다.
현재 사용중인 lgbm version은 3.1.1 입니다.


# EDA

## train data

In [ ]:
## service 편집
train_service = pd.read_csv("train_service.csv")

#user별 최신 등록일 표기(중복 가입 사례 제거)
train_service_drop_use = train_service.sort_values(['user_id', 'reg_date'])['user_id'].drop_duplicates(keep='last')
train_service2 = train_service.iloc[train_service_drop_use.index].reset_index(drop=True)

# 미국 달러 결제자에 대한 환전 작업
# dollar = []
for i in range(train_service2.shape[0]):
    if str(train_service2['pgamount'][i])[-1] == '9':
#         dollar.append('dollar')
        train_service2['pgamount'][i] = round(train_service2['pgamount'][i],2) * 1100
    else:
#         dollar.append('korean')

# train_service2['dollar'] = dollar
train_service2['pgamount'] = [round(i, -2) for i in train_service2['pgamount']] #소수점 제거

# 20대 미만, 20대 ~ 50대, 60대 이상 나이 통합
train_service2 = train_service2[train_service2['agegroup'] <= 100]
train_service2['agegroup'] = [str(i) for i in train_service2['agegroup']]
train_service2.loc[(train_service2['agegroup'] <= '15') |
                   (train_service2['agegroup'] == '5'), 'agegroup'] = '20대 미만'
train_service2.loc[(train_service2['agegroup'] == '20') | 
                   (train_service2['agegroup'] == '25'), 'agegroup'] = '20대'
train_service2.loc[(train_service2['agegroup'] == '30') | 
                   (train_service2['agegroup'] == '35'), 'agegroup'] = '30대'
train_service2.loc[(train_service2['agegroup'] == '40') | 
                   (train_service2['agegroup'] == '45'), 'agegroup'] = '40대'
train_service2.loc[(train_service2['agegroup'] == '50') |
                   (train_service2['agegroup'] == '55'), 'agegroup'] = '50대'
train_service2.loc[(train_service2['agegroup'] >= '60'), 'agegroup'] = '60대 이상'


# 가구내 인구수 3이면 4로 통합
train_service2.loc[(train_service2['concurrentwatchcount'] == 3),
                   'concurrentwatchcount'] = 4

# 결제 코드 변환
train_service2['chargetypeid'] = [str(i) for i in train_service2['chargetypeid']]
train_service2.loc[(train_service2['chargetypeid'] == '121') |
                   (train_service2['chargetypeid'] == '131'),
                   'chargetypeid'] = '결제_휴대폰'
train_service2.loc[(train_service2['chargetypeid'] == '132') |
                   (train_service2['chargetypeid'] == '134'),
                   'chargetypeid'] = '결제_신용카드'
train_service2.loc[(train_service2['chargetypeid'] == '140'),
                   'chargetypeid'] = '결제_아이튠즈'
train_service2.loc[(train_service2['chargetypeid'] == '151'),
                   'chargetypeid'] = '결제_원스토어'
train_service2.loc[(train_service2['chargetypeid'] == '160') |
                   (train_service2['chargetypeid'] == '170') |
                   (train_service2['chargetypeid'] == '180') |
                   (train_service2['chargetypeid'] == '190'),
                   'chargetypeid'] = '결제_페이서비스'

#등록 기기 타입 변환
train_service2.loc[(train_service2['dev_typeid'] == 'mobile'),
                   'dev_typeid'] = '결제기기_mobile_web'
train_service2.loc[train_service2['dev_typeid'] == 'pc',
                   'dev_typeid'] = '결제기기_pc'
train_service2.loc[train_service2['dev_typeid'] == 'ios', 
                   'dev_typeid'] = '결제기기_ios'
train_service2.loc[train_service2['dev_typeid'] == 'android',
                   'dev_typeid'] = '결제기기_android'
train_service2.loc[(train_service2['dev_typeid'] != 'movile web') &
                   (train_service2['dev_typeid'] != 'android') &
                   (train_service2['dev_typeid'] != 'ios') &
                   (train_service2['dev_typeid'] != 'pc'),
                   'dev_typeid'] = '결제기기_스마트_tv_관련'

# user_id별 가장 오래된, 최신 시청 시간 확인 후, 갭 차이 변수화
min_max_bookmark = train_bookmark.groupby(['user_id'
                                           ])['dates'].agg(['max', 'min'
                                                            ]).reset_index()
min_max_bookmark['max'] = [
    datetime.strptime(min_max_bookmark['max'][i], '%Y-%m-%d')
    for i in range(min_max_bookmark.shape[0])
]

min_max_bookmark['min'] = [
    datetime.strptime(min_max_bookmark['min'][i], '%Y-%m-%d')
    for i in range(min_max_bookmark.shape[0])
]

min_max_bookmark['day_gap'] = [
    int(
        str(min_max_bookmark['max'][i] -
            min_max_bookmark['min'][i]).split(' ')[0])
    for i in range(min_max_bookmark.shape[0])
]

train_service2 = pd.merge(train_service2,
                          min_max_bookmark[['user_id', 'day_gap']],
                          how='left',
                          on='user_id')
train_service2.loc[train_service2.isnull()['day_gap'], 'day_gap'] = 0

## bookmark 편집
train_bookmark = pd.read_csv("train_bookmark.csv")

# 각 컨텐츠 별, 시청 기기 변환
train_bookmark['dev_type'] = [str(i) for i in train_bookmark['dev_type']]

train_bookmark.loc[(train_bookmark['dev_type'] == '1'), 'dev_type'] = 'pc'
train_bookmark.loc[(train_bookmark['dev_type'] == '2') |
                   (train_bookmark['dev_type'] == '3'), 'dev_type'] = 'android'
train_bookmark.loc[(train_bookmark['dev_type'] == '4') |
                   (train_bookmark['dev_type'] == '5'), 'dev_type'] = 'ios'
train_bookmark.loc[(train_bookmark['dev_type'] != 'pc') &
                   (train_bookmark['dev_type'] != 'android') &
                   (train_bookmark['dev_type'] != 'ios'),
                   'dev_type'] = '스마트 tv 관련'

## 공통 데이터

In [ ]:
## 코인 이벤트 관련 데이터
copper = pd.read_csv("copper.csv")
copper = copper.sort_values(['user_id','reg_date']).reset_index(drop=True)
copper['co'] = [copper['prod_code'][i][0:2] for i in range(copper.shape[0])]

abc = DataFrame(Counter(copper['prod_code']).values(), Counter(copper['prod_code']).keys()).reset_index()
abc['ab'] = [abc['index'][i][0:2] for i in range(abc.shape[0])]

#컨텐츠 관련 데이터
컨텐츠_설명 = pd.read_csv("컨텐츠_설명.csv")
컨텐츠_설명 = 컨텐츠_설명[['cont_id', 'playtime', 'broaddate']]

영화_설명 = pd.read_csv("영화_설명.csv")
영화_설명.columns = [
    'cont_id', 'title_ko', 'title_en', 'region', 'director', 'actor',
    'targetage', '설명', 'run_time'
]
영화_설명 = 영화_설명[['cont_id', 'region', 'targetage', 'run_time']]

abc = []
for i in range(영화_설명.shape[0]):
    try:
        if 영화_설명['run_time'][i][-1] == '분':
            abc.append(영화_설명['run_time'][i][:-1])
        else:
            abc.append(영화_설명['run_time'][i])
    except:
        abc.append(영화_설명['run_time'][i])

영화_설명['run_time'] = [float(i) * 60 for i in abc]

## (통합버전)각 시청 기록에, content, movie 관련 정보 합치기
train_content_bookmark = pd.merge(train_bookmark,
                                  컨텐츠_설명,
                                  on='cont_id',
                                  how='left')
train_total_bookmark = pd.merge(train_content_bookmark,
                                영화_설명,
                                on='cont_id',
                                how='left')

coerce_df_columns_to_numeric(train_total_bookmark, 'playtime')

In [ ]:
#드라마 playtime, 영화 run_time 합치기

list1 = []
for i in tqdm(range(train_total_bookmark.shape[0])):
    if np.isnan(train_total_bookmark['playtime'][i]) == True:
        list1.append(train_total_bookmark['run_time'][i])
    else:
        list1.append(train_total_bookmark['playtime'][i])

#각 컨텐츠별 시간과 개인의 시청별 시간으로 view_percent를 추가 생성
train_total_bookmark['content_time'] = list1
train_total_bookmark.loc[train_total_bookmark['content_time'] == 0,
                         ['content_time']] = np.nan
train_total_bookmark['view_percent'] = train_total_bookmark[
    'viewtime'] / train_total_bookmark['content_time']

train_total_bookmark2 = train_total_bookmark.copy()
train_total_bookmark2.loc[train_total_bookmark2['view_percent'] >= 1,
                          ['view_percent']] = 1.00

train_total_bookmark2.loc[train_total_bookmark2['channeltype'] == 'E',
                          'channeltype'] = 'L'

# 드라마, 영화 공개일과 시청일간의 차이를 계산 후, 변수 추가함
drama_view_gap = []
for i in tqdm(range(train_total_bookmark2.shape[0])):
    try:
        drama_view_gap.append(
            (datetime.strptime(train_total_bookmark2['dates'][i], '%Y-%m-%d') -
             datetime.strptime(train_total_bookmark2['broaddate'][i],
                               '%Y-%m-%d')).days)
    except:
        drama_view_gap.append('해당 없음')

train_total_bookmark2['drama_view_day_gap'] = drama_view_gap
train_total_bookmark2.loc[train_total_bookmark2['drama_view_day_gap'] ==
                          '해당 없음', 'drama_view_day_gap'] = -9999

# 7일 이하를 최신 기간, 1년 미만을 중간 기간, 1년 이상과 영화(해당 없음)으로 추가 그룹화 수행
temp1 = train_total_bookmark2[train_total_bookmark2['drama_view_day_gap'] ==
                              -9999]
temp1['drama_view_day_gap'] = '해당 없음'

temp2 = train_total_bookmark2[
    train_total_bookmark2['drama_view_day_gap'] >= 365]
temp2['drama_view_day_gap'] = '1년 이상'

temp3 = train_total_bookmark2[
    (train_total_bookmark2['drama_view_day_gap'] > 7)
    & (train_total_bookmark2['drama_view_day_gap'] < 365)]
temp3['drama_view_day_gap'] = '중간 기간'

temp4 = train_total_bookmark2[
    (train_total_bookmark2['drama_view_day_gap'] >= 0)
    & (train_total_bookmark2['drama_view_day_gap'] <= 7)]
temp4['drama_view_day_gap'] = '최신 기간'

train_total_bookmark3 = pd.concat([temp1, temp2, temp3, temp4],
                                  axis=0).reset_index(drop=True)

In [ ]:
#채널 타입과 user별 시청수 결합 후, 특정 이상치 절삭
DataFrame(
    train_total_bookmark3.groupby([
        'user_id', 'channeltype'
    ])['dates'].count()).unstack().reset_index().to_csv('temp_channeltype.csv')

content_count = pd.read_csv('temp_channeltype.csv', header=1)
del content_count['channeltype']
content_count.columns = ['user_id', 'ETC', 'MOVIE', 'VIEW_DRAMA']

content_count.loc[content_count['VIEW_DRAMA'] >= 400, 'VIEW_DRAMA'] = 400
content_count.loc[content_count['MOVIE'] >= 40, 'MOVIE'] = 40
content_count.loc[content_count['ETC'] >= 40, 'ETC'] = 40
content_count = content_count.fillna(0)

train_total = pd.merge(train_service2, content_count, how='left', on='user_id')

#드라마 시청 갭과 user별 시청수 결합 후, 특정 이상치 절삭
DataFrame(
    train_total_bookmark3.groupby([
        'user_id', 'drama_view_day_gap'
    ])['dates'].count()).unstack().reset_index().to_csv(
        'temp_drama_view_day_gap.csv')

drama_view_day_gap = pd.read_csv('temp_drama_view_day_gap.csv', header=1)
del drama_view_day_gap['drama_view_day_gap']
drama_view_day_gap.columns = ['user_id'] + list(drama_view_day_gap.columns[1:])
drama_view_day_gap = drama_view_day_gap.fillna(0)
drama_view_day_gap.loc[drama_view_day_gap['1년 이상'] >= 100, '1년 이상'] = 100
drama_view_day_gap.loc[drama_view_day_gap['중간 기간'] >= 100, '중간 기간'] = 100
drama_view_day_gap.loc[drama_view_day_gap['최신 기간'] >= 100, '최신 기간'] = 100
drama_view_day_gap.loc[drama_view_day_gap['해당 없음'] >= 100, '해당 없음'] = 100

train_total22 = pd.merge(train_total,
                         drama_view_day_gap,
                         how='left',
                         on='user_id')

#유저별, 평균 시청 퍼센트 계산
view_percent = DataFrame(
    train_total_bookmark3.groupby(['user_id'
                                   ])['view_percent'].mean()).reset_index()

train_total2 = pd.merge(train_total22, view_percent, how='left', on='user_id')
train_total2.loc[train_total2.isnull()['view_percent'], ['view_percent']] = 0

# 시각화 부문 미 실행 처리
# plt.hist(train_total2['view_percent'],bins=50);
# plt.hist(train_total2['view_count'],bins=50);

## test data

(train과 동일한 구조로 코드 생략 처리)

## 재 구독율 관련 counter 계산 및 추가 시각화

(데이터 미 공개로 전체 삭제)

# 모델링

## 모델링 전 데이터 최종 정리

In [ ]:
# 전 처리된 데이터 불러오기

# train = pd.read_csv("train_eda_ver9.csv")
train_set2 = train_total2.copy()

#user_id, 날짜, 추가 미사용 변수 제거
del train_set2['user_id']
del train_set2['reg_date']
del train_set2['enddate']
del train_set2['prod_code']
del train_set2['promo_100']

# 반응 변수 추가 제거
del train_set2['Repurchase']

#train_y(반응변수만 있는 데이터) 생성
train_label = pd.get_dummies(train_set2['Repurchase'])
train_label = train_label['X']

#category 변수 dummy화 선언(train)
chargetypeid_dummy = pd.get_dummies(train_set2['chargetypeid'])
del train_set2['chargetypeid']

copperReceived_dummy = pd.get_dummies(train_set2['copperReceived'],
                                      dummy_na=True)
copperReceived_dummy.columns = ['copperReceived_yes', 'copperReceived_no']
del copperReceived_dummy['copperReceived_no']
del train_set2['copperReceived']

dev_typeid_dummy = pd.get_dummies(train_set2['dev_typeid'], dummy_na=True)
del train_set2['dev_typeid']

isauth_dummy = pd.get_dummies(train_set2['isauth'], dummy_na=True)
isauth_dummy.columns = ['isauth_yes', 'isauth_no']
del isauth_dummy['isauth_no']
del train_set2['isauth']

gender_dummy = pd.get_dummies(train_set2['gender'], dummy_na=True)
gender_dummy.columns = ['F', 'M', '성별 모름']
del train_set2['gender']

agegroup_dummy = pd.get_dummies(train_set2['agegroup'])
del train_set2['agegroup']

train_set = pd.concat([
    train_set2, chargetypeid_dummy, copperReceived_dummy, dev_typeid_dummy,
    isauth_dummy, gender_dummy, agegroup_dummy
], axis=1)

#category 변수 dummy화 선언(test) 생략함

## using gridsearch

In [ ]:
NUM_BOOST_ROUND = 10000
n_splits =5
SEED = 1993

lgbm_param = {'objective':'binary',
              'boosting_type': 'gbdt',
              'random_state':1993,
              'learning_rate':0.01,
              'subsample':0.8,
            "metric" : "auc",
            'subsample_freq': 1,
              'reg_lambda':0.1,
              'reg_alpha': 0.2,
               'num_leaves': (2**4)-1,
              'colsample_bytree':0.9,
            }

cv_list = []
evals_result = {}
real_train_f1, real_valid_f1 = [], []

kfolds = StratifiedKFold(n_splits=n_splits, random_state=SEED, shuffle=True)

cv_subsample = [0.8,0.9]
cv_reg_lambda = [0.1,0.2]
cv_reg_alpha = [0.2,0.4]
cv_num_leaves = [8,16]
cv_colsample_bytree = [0.8,0.9]

total_items2 = [cv_subsample,cv_reg_lambda,cv_reg_alpha,
              cv_num_leaves,cv_colsample_bytree]

total_items = list(product(*total_items2))

In [ ]:
for jk in tqdm_notebook(range(len(total_items))):
    one_items = total_items[jk]
    
    lgbm_param['subsample'] = one_items[0]
    lgbm_param['reg_lambda'] = one_items[1]
    lgbm_param['reg_alpha'] = one_items[2]
    lgbm_param['num_leaves'] = one_items[3]
    lgbm_param['colsample_bytree'] = one_items[4] 
    
    train_f1, valid_f1 = [], []

    for ind, (trn_ind, val_ind) in tqdm_notebook( enumerate(kfolds.split(X= train_set, y = train_label)) ):

        X_train, y_train = train_set.iloc[trn_ind], train_label[trn_ind]
        X_valid, y_valid = train_set.iloc[val_ind], train_label[val_ind]

        dtrain = lgbm.Dataset( X_train, y_train )
        dvalid = lgbm.Dataset( X_valid, y_valid ,reference=dtrain)

        model = lgbm.train(lgbm_param , dtrain, NUM_BOOST_ROUND, valid_sets=(dtrain, dvalid), valid_names=('train','valid'), 
                            verbose_eval= 100, evals_result=evals_result, early_stopping_rounds=50)

        train_x_predict = model.predict(X_train)
        train_x_predict_binary = [1 if i>=0.29 else 0 for i in train_x_predict]
        train_f1.append(f1_score(y_train, train_x_predict_binary))

        valid_x_predict = model.predict(X_valid)
        valid_x_predict_binary = [1 if i>=0.29 else 0 for i in valid_x_predict]
        valid_f1.append(f1_score(y_valid, valid_x_predict_binary))

        print('='*80)
    
    real_train_f1.append(np.mean(train_f1))
    real_valid_f1.append(np.mean(valid_f1))

In [ ]:
checking_value = DataFrame(total_items)
checking_value.columns = ['subsample','reg_lambda','reg_alpha',
                  'num_leaves','colsample_bytree']
checking_value['train_f1'] = real_train_f1
checking_value['valid_f1'] = real_valid_f1
# checking_value.sort_values('valid_f1',ascending=False)

## grid search 결과로 5-fold + threshold 찾기 수행

In [ ]:
lgbm_param = {'objective':'binary',
              'boosting_type': 'gbdt',
              'random_state':1993,
              'learning_rate':0.01,
              'subsample':0.9,
            "metric" : "auc",
            'subsample_freq': 1,
              'reg_lambda':0.1,
              'reg_alpha': 0.2,
               'num_leaves': 8,
              'colsample_bytree':0.9
            }

cv_list = []
evals_result = {}
train_f1 = []
valid_f1 = []
final_test = np.zeros( test.shape[0] )

kfolds = StratifiedKFold(n_splits=n_splits, random_state=SEED, shuffle=True)

for ind, (trn_ind, val_ind) in tqdm_notebook( enumerate(kfolds.split(X= train_set, y = train_label)) ):
    
    X_train, y_train = train_set.iloc[trn_ind], train_label[trn_ind]
    X_valid, y_valid = train_set.iloc[val_ind], train_label[val_ind]
    
    dtrain = lgbm.Dataset( X_train, y_train )
    dvalid = lgbm.Dataset( X_valid, y_valid ,reference=dtrain)
    
    model = lgbm.train(lgbm_param , dtrain, NUM_BOOST_ROUND, valid_sets=(dtrain, dvalid), valid_names=('train','valid'), 
                        verbose_eval= 100, evals_result=evals_result, early_stopping_rounds=50)
    
    train_x_predict = model.predict(X_train)
    train_x_predict_binary = [1 if i>=0.29 else 0 for i in train_x_predict]
    train_f1.append(f1_score(y_train, train_x_predict_binary))

    valid_x_predict = model.predict(X_valid)
    valid_x_predict_binary = [1 if i>=0.29 else 0 for i in valid_x_predict]
    valid_f1.append(f1_score(y_valid, valid_x_predict_binary))
    
    test_pred  = model.predict(test_set)
    final_test += test_pred

    print('='*80)
    
# np.mean(train_f1), np.mean(valid_f1)

In [ ]:
thres_train, thres_valid = [], []

for thres in tqdm([round(i,3) for i in list(np.arange(0.01, 0.99, 0.005))]):
    train_x_predict_binary = [1 if i>=thres else 0 for i in train_x_predict]
    thres_train.append(f1_score(y_train, train_x_predict_binary))
    valid_x_predict_binary = [1 if i>=thres else 0 for i in valid_x_predict]
    thres_valid.append(f1_score(y_valid, valid_x_predict_binary))

checking = DataFrame([[round(i,3) for i in list(np.arange(0.01, 0.99, 0.005))], thres_train, thres_valid]).T
checking.columns = ['threshold','train_f1','valid_f1']
# checking.sort_values('valid_f1',ascending=False).head(10)

best_threshold = float(checking.sort_values('valid_f1',ascending=False)['threshold'].head(1))

train_x_predict = model.predict(X_train)
train_x_predict_binary = [1 if i>= best_threshold else 0 for i in train_x_predict]

# print(Counter(train_x_predict_binary), Counter(y_train))
# accuracy_score(y_train, train_x_predict_binary), f1_score(y_train, train_x_predict_binary)

valid_x_predict = model.predict(X_valid)
valid_x_predict_binary = [1 if i>= best_threshold else 0 for i in valid_x_predict]

# print(Counter(valid_x_predict_binary), Counter(y_valid))
# accuracy_score(y_valid, valid_x_predict_binary),f1_score(y_valid, valid_x_predict_binary)

# fig, ax = plt.subplots(figsize=(16, 12))
# lgbm.plot_importance(model, ax=ax)

# confusion_matrix(y_train, train_x_predict_binary)

# print(classification_report(y_train, train_x_predict_binary))
# print(classification_report(y_valid, valid_x_predict_binary))

In [ ]:
real_final = final_test / 5
test_predict = [1 if i>=best_threshold else 0 for i in real_final]

test = pd.read_csv("test_eda_ver7.csv")
test['value'] = test_predict

submission = pd.read_csv("C:\\Users\\bluemumin\\Desktop\\CDS\\Predict\\CDS_submission.csv")
submission['user_id'] = [i.split('|')[0] for i in submission['KEY']]
submission.head()

real_submission = pd.merge(submission, test[['user_id','value']], how='left', on='user_id', indicator=True)

real_submission2 = real_submission[['KEY','value']]
real_submission2.columns = ['KEY','CHURN']

real_submission2.to_csv("wavve_재가입여부_분류결과.csv", index=False)

# 적용 후 취소 항목

## 데이터 전처리

In [ ]:
# # 2-1 공통 데이터 중 결제 관련 데이터

# product_code2 = pd.read_csv("자동결제.csv")

# check_auto = []
# for i in range(product_code2.shape[0]):
#     try:
#         if product_code2['설명'][i].find('자동결제') >= 0:
#             check_auto.append('자동결제')
#         elif product_code2['설명'][i].find('미사용') >= 0:
#             check_auto.append('미사용')
#         else:
#             check_auto.append('수동결제')
#     except:
#         check_auto.append('미사용')
# product_code2['check_auto'] = check_auto

# mobile_auto = []
# for i in range(product_code2.shape[0]):
#     try:
#         if product_code2['설명'][i].find('모바일') >= 0:
#             mobile_auto.append('모바일 가능')
#         else:
#             mobile_auto.append('없음')
#     except:
#         mobile_auto.append('없음')
# product_code2['mobile_auto'] = mobile_auto

# pc_auto = []
# for i in range(product_code2.shape[0]):
#     try:
#         if product_code2['설명'][i].find('PC') >= 0:
#             pc_auto.append('PC 가능')
#         else:
#             pc_auto.append('없음')
#     except:
#         pc_auto.append('없음')
# product_code2['pc_auto'] = pc_auto

# tv_auto = []
# for i in range(product_code2.shape[0]):
#     try:
#         if product_code2['설명'][i].find('스마트TV') >= 0:
#             tv_auto.append('스마트TV 가능')
#         else:
#             tv_auto.append('없음')
#     except:
#         tv_auto.append('없음')
# product_code2['tv_auto'] = tv_auto

# all_auto = []
# for i in range(product_code2.shape[0]):
#     try:
#         if product_code2['설명'][i].find('전체 디바이스') >= 0:
#             all_auto.append('전체 가능')
#         else:
#             all_auto.append('없음')
#     except:
#         all_auto.append('없음')
# product_code2['all_auto'] = all_auto

# product_code2.loc[product_code2['all_auto'] != '없음', 'mobile_auto'] = '모바일 가능'
# product_code2.loc[product_code2['all_auto'] != '없음', 'pc_auto'] = 'PC 가능'
# product_code2.loc[product_code2['all_auto'] != '없음', 'tv_auto'] = '스마트TV 가능'
# del product_code2['all_auto']

# hd_auto = []
# for i in range(product_code2.shape[0]):
#     try:
#         if product_code2['설명'][i].find(' HD 화질') >= 0:
#             hd_auto.append('HD 가능')
#         else:
#             hd_auto.append('없음')
#     except:
#         hd_auto.append('없음')
# product_code2['hd_auto'] = hd_auto

# fhd_auto = []
# for i in range(product_code2.shape[0]):
#     try:
#         if product_code2['설명'][i].find('FHD 화질') >= 0:
#             fhd_auto.append('FHD 가능')
#         else:
#             fhd_auto.append('없음')
#     except:
#         fhd_auto.append('없음')
# product_code2['fhd_auto'] = fhd_auto

# good_auto = []
# for i in range(product_code2.shape[0]):
#     try:
#         if product_code2['설명'][i].find('최상위 화질') >= 0:
#             good_auto.append('최상위 가능')
#         else:
#             good_auto.append('없음')
#     except:
#         good_auto.append('없음')
# product_code2['good_auto'] = good_auto

# product_code2.columns = ['prod_code', 'name', '설명'] + list( product_code2.columns[3:])
# product_code = product_code2.copy()
# del product_code['name']
# del product_code['설명']

# train_service2 = pd.merge(train_service2,
#                           product_code,
#                           how='left',
#                           on='prod_code')

In [ ]:
# 2-2 train data 시청 시간대
#bookmark_device = train_bookmark.groupby(['user_id', 'dev_type'
#                                           ])['dates'].count().reset_index()
# bookmark_device2 = pd.pivot_table(bookmark_device,
#                                   index=['user_id'],
#                                   values='dates',
#                                   columns='dev_type').reset_index()
# bookmark_device2 = bookmark_device2.fillna(0)
# bookmark_device2.columns = ['user_id', 'view_android', 'view_ios', 'view_pc', 'view_smart_tv']

# bookmark_device2['best'] = bookmark_device2[list(
#     bookmark_device2.columns[1:])].idxmax(axis=1)

# train_bookmark.loc[(train_bookmark['sector'] == '1') |
#                    (train_bookmark['sector'] == 'M01'), 'sector'] = '드라마관련'
# train_bookmark.loc[(train_bookmark['sector'] == '2') |
#                    (train_bookmark['sector'] == 'M03'), 'sector'] = '예능관련'
# train_bookmark.loc[(train_bookmark['sector'] == '3'), 'sector'] = '시사관련'
# train_bookmark.loc[(train_bookmark['sector'] == '8') |
#                    (train_bookmark['sector'] == '6') |
#                    (train_bookmark['sector'] == 'M07'), 'sector'] = '애니관련'
# train_bookmark.loc[(train_bookmark['sector'] == '9') |
#                    (train_bookmark['sector'] == '10'), 'sector'] = '해외관련'
# train_bookmark.loc[(train_bookmark['sector'] == 'M02'), 'sector'] = '로맨스관련'
# train_bookmark.loc[(train_bookmark['sector'] == 'M04') |
#                    (train_bookmark['sector'] == 'M05') |
#                    (train_bookmark['sector'] == 'M06'), 'sector'] = '스릴러관련'
# train_bookmark.loc[(train_bookmark['sector'] == '4') |
#                    (train_bookmark['sector'] == '5') |
#                    (train_bookmark['sector'] == '7') |
#                    (train_bookmark['sector'] == '11') |
#                    (train_bookmark['sector'] == '12') |
#                    (train_bookmark['sector'] == 'M08') |
#                    (train_bookmark['sector'] == 'M90'), 'sector'] = '기타관련'
# train_bookmark.loc[train_bookmark.isnull()['sector'], 'sector'] = '확인불가sector'

# bookmark_sector = train_bookmark.groupby(['user_id', 'sector'
#                                           ])['dates'].count().reset_index()
# bookmark_sector2 = pd.pivot_table(bookmark_sector,
#                                   index=['user_id'],
#                                   values='dates',
#                                   columns='sector').reset_index()
# bookmark_sector2 = bookmark_sector2.fillna(0)
# bookmark_sector2['sector_best'] = bookmark_sector2[list(
#     bookmark_sector2.columns[1:])].idxmax(axis=1)


# time_bookmark = train_bookmark.groupby(['user_id'])['hour'].agg(['mean']).reset_index()
# time_bookmark.columns = ['user_id', '시청시간_mean']
# time_bookmark['시청시간_mean'] = [round(i, 0) for i in time_bookmark['시청시간_mean']]


# train_service2 = pd.merge(train_service2,
#                           time_bookmark[['user_id', '시청시간_mean']],
#                           how='left',
#                           on='user_id')
# train_service2.loc[train_service2.isnull()['시청시간_mean'], '시청시간_mean'] = 0

# train_service2.loc[(train_service2['시청시간_mean'] >= 22) |
#                    (train_service2['시청시간_mean'] <= 7), '시청시간_mean'] = -99
# train_service2.loc[(train_service2['시청시간_mean'] >= 8) &
#                    (train_service2['시청시간_mean'] <= 12), '시청시간_mean'] = 9
# train_service2.loc[(train_service2['시청시간_mean'] >= 13) &
#                    (train_service2['시청시간_mean'] <= 17), '시청시간_mean'] = 15
# train_service2.loc[(train_service2['시청시간_mean'] >= 18) &
#                    (train_service2['시청시간_mean'] <= 21), '시청시간_mean'] = 20

# train_service2.loc[(train_service2['시청시간_mean'] == -99), '시청시간_mean'] = "심야시간"
# train_service2.loc[(train_service2['시청시간_mean'] == 9), '시청시간_mean'] = "아침시간"
# train_service2.loc[(train_service2['시청시간_mean'] == 15), '시청시간_mean'] = "점심시간"
# train_service2.loc[(train_service2['시청시간_mean'] == 20), '시청시간_mean'] = "저녁시간"


# train_total3 = pd.merge(train_total2,
#                         bookmark_device2[['user_id', 'best']],
#                         how='left',
#                         on='user_id')
# train_total3.loc[train_total3.isnull()['best'], 'best'] = 'not watch'

# train_total4 = pd.merge(train_total3,
#                         bookmark_sector2[['user_id', 'sector_best']],
#                         how='left',
#                         on='user_id')
# train_total4.loc[train_total4.isnull()['sector_best'],
#                  'sector_best'] = '미시청sector'

## rf시도

In [ ]:
# train_set = train_set.fillna(0)
# test_set = test_set.fillna(0)

# NUM_BOOST_ROUND = 10000
# n_splits =5
# SEED = 1993
# threshold = 0.28

# kfolds = StratifiedKFold(n_splits=n_splits, random_state=SEED, shuffle=True)

# rf = RandomForestClassifier(n_estimators=500, criterion='gini', max_depth=None, min_samples_split=5, min_samples_leaf=5,
#                            random_state = 1217,  max_features='auto')
# rf_final_test = np.zeros( test_set.shape[0] )

# for ind, (trn_ind, val_ind) in tqdm_notebook( enumerate(kfolds.split(X= train_set, y = train_label)) ):
    
#     X_train, y_train = train_set.iloc[trn_ind], train_label[trn_ind]
#     X_valid, y_valid = train_set.iloc[val_ind], train_label[val_ind]
    
#     rf.fit(X_train, y_train)
    
#     train_predicted_proba = rf.predict_proba(X_train)
#     train_predicted = (train_predicted_proba [:,1] >= threshold).astype('int')

#     print(accuracy_score(y_train, train_predicted), f1_score(y_train, train_predicted))
    
#     valid_predicted_proba = rf.predict_proba(X_valid)
#     valid_predicted = (valid_predicted_proba [:,1] >= threshold).astype('int')

#     print(accuracy_score(y_valid, valid_predicted), f1_score(y_valid, valid_predicted))
    
#     test_pred  = rf.predict_proba(test_set)[:,1]
#     rf_final_test += test_pred
    
#     print("="*80)

# test_pred_proba  = rf.predict_proba(test_set)[:,1]
# test_pred = (test_pred >= threshold).astype('int')
# # Counter(test_pred)

## xgboost시도

In [ ]:
# xgb_clf = XGBClassifier(
#     learning_rate =0.01,
#     n_estimators=1000,
#     min_child_weight=1,
#     gamma=0,
#     subsample=0.9,
#     colsample_bytree=0.9,
#     objective= 'binary:logistic',
#     nthread=-1,
#     seed=1993
# )

# kfolds = StratifiedKFold(n_splits=n_splits, random_state=SEED, shuffle=True)
# xgb_final_test = np.zeros( test_set.shape[0] )

# for ind, (trn_ind, val_ind) in tqdm_notebook( enumerate(kfolds.split(X= train_set, y = train_label)) ):
    
#     X_train, y_train = train_set.iloc[trn_ind], train_label[trn_ind]
#     X_valid, y_valid = train_set.iloc[val_ind], train_label[val_ind]

#     xgb_clf.fit(X_train, y_train, eval_metric='aucpr')

#     train_predicted_proba = xgb_clf.predict_proba(X_train)
#     train_predicted = (train_predicted_proba [:,1] >= threshold).astype('int')

#     print(accuracy_score(y_train, train_predicted), f1_score(y_train, train_predicted))
    
#     valid_predicted_proba = xgb_clf.predict_proba(X_valid)
#     valid_predicted = (valid_predicted_proba [:,1] >= threshold).astype('int')

#     print(accuracy_score(y_valid, valid_predicted), f1_score(y_valid, valid_predicted))
    
#     test_pred  = xgb_clf.predict_proba(test_set)[:,1]
#     xgb_final_test += test_pred
    
#     print("="*80)

# test_pred_proba  = xgb_clf.predict_proba(test_set)[:,1]
# test_pred = (test_pred >= threshold).astype('int')

# # Counter(test_pred)